In [1]:
import pathlib
import numpy as np
import torch
from datasets import load_dataset

In [3]:
mask=torch.tril(torch.ones(5,5))
print(f"mask size : {mask.size()}")

mask size : torch.Size([5, 5])


In [ ]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
# Load the full dataset
import pathlib
datasets_folder = pathlib.Path(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition\data")
wine_review_filepath=datasets_folder/"wine_reviews"/"winemag-data-130k-v2.json"
data = load_dataset(str(wine_review_filepath.parent),'json')

In [ ]:
data

In [ ]:
data['train']['description'][:3]

In [ ]:
train_ds= data['train']

In [ ]:
countries=train_ds['country']
print(f"there are {len(set(countries))} countries exist in wine review dataset")

In [ ]:
varieties = train_ds['variety']
print(f"there are {len(set(varieties))} varieties in wine review dataset")

In [ ]:
def prepare_text_with_country_variety(batch):
    text_with_country_variety = [f"{country} : {province} : {variety} : {description}" for country, province, variety, description in zip(batch['country'],batch['province'], batch['variety'], batch['description'])]
    return {"text": text_with_country_variety}

In [ ]:
wine_ds = data.map(prepare_text_with_country_variety,batched=True, batch_size=None)

In [ ]:
train_ds = wine_ds["train"]

In [ ]:
df=train_ds.to_pandas()

In [ ]:
df[(df.country=='US')&(df.province=='California')]['variety'].value_counts()

In [ ]:
chardonnay_descriptions=df[(df.country=='US')&(df.province=='California')&(df.variety=='Chardonnay')]['description']


In [ ]:
chardonnay_descriptions.index=range(len(chardonnay_descriptions))

In [ ]:
chardonnay_descriptions[chardonnay_descriptions.map(lambda description : "vines" in description)][176]

In [ ]:
wine_ds["train"]["text"][:2]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
model_chkpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_chkpt)

In [ ]:
tokens = tokenizer(wine_ds["train"]["text"][1])

In [ ]:
tokens

In [ ]:
text_tokens = {input_id : tokenizer.decode(input_id) for input_id in tokens["input_ids"]}

In [ ]:
text_tokens